![Portada Pregunta 3](Pregunta_3_Imagen.png)

Considere que, apoyándose en el desarrollo de la Ciencia de Datos y la Optimización avanzada, usted desea
conformar un nuevo equipo de fútbol. Para esto, usted ha logrado recopilar información sobre 325 futbolistas.
Esta información corresponde a la valoración que se tiene, para cada uno de ellos, de las 4 posiciones en
el campo de juego: Arquero, Defensa, Mediocampista y Delantero; y al precio de contratar a ese jugador.
Esta valoración está medida en un coeficiente que va desde el 0 al 100, donde el 100 es la mayor valoración.
Además, cuanta con la información de qué jugadores no debeŕıan compartir equipo con qué jugadores. Esta
información está clasificada como 0 (pueden compartir equipo), 1 (conflicto 1: preferentemente no debeŕıan
compartir equipo) y 2 (conflicto 2: no debeŕıan compartir equipo). Toda esta información se encuentra en el
archivo Datos.xls, en la hojas Caracteŕısticas y Conflictos.

Basado en esta información, se le pide conformar un equipo con la mayor valoración posible que cuente con
33 jugadores (no se puede contratar a un jugador en más de una posición), de los cuales 3 son Arqueros, 10
Defensas, 12 son Mediocampistas y 8 son Delanteros, y que se limite por el presupuesto disponible, que es de
22000 pesos. Para esto, usted sabe que cada jugador de la lista puede ser contratado en cualquier de las 4
posiciones, que la valoración total del equipo es la suma de las valoraciones individuales de la posición escogida
para cada jugador contratado, y que el costo total de la plantilla de jugadores es la suma de los precios de
contratación de todos los jugadores contratados.

#### Nota; este proyecto ocupa NumPy, Pandas y Gurobi. Si no tiene alguno de estos corra la siguiente celda para instalarlos. De no ser así se puede saltar la celda. 

In [1]:
!pip install numpy
!pip install pandas
!pip install gurobipy


*(a) (10 puntos) Diseñe un algoritmo, prográmelo y ejecútelo en Python, para seleccionar aleatoriamente jugadores a cada una de las 33 posiciones requeridas sin sobrepasar el presupuesto disponible. Registre la solución en: jugadores seleccionados y su posición, valoración total del equipo, total de conflictos (separados en conflictos 1 y conflictos 2) y listados de casos (jugadores en cada conflicto), y el costo total de la plantilla. Una vez diseñado este algoritmo, ejecútelo 1000 veces, y entregue la solución que tiene menos puntaje en conflictos (sume 1 punto a cada conflicto 1 y 5 puntos a cada conflicto 2). En caso de que haya más de una solución con el mismo puntaje en conflictos, escoja aquella que tenga mayor valoración total.*

In [2]:
import random
import pandas as pd
import time

In [3]:
## Definimos variables 
# Número de jugadores
num_jugadores = 325

# Número de posiciones
num_posiciones = 4

# Número de jugadores por posición
num_arqueros = 3
num_defensas = 10
num_mediocampistas = 12
num_delanteros = 8

# Presupuesto disponible
presupuesto = 22000

datos = pd.read_excel("Datos.xlsx", sheet_name="Características")
conflictos = pd.read_excel("Datos.xlsx", sheet_name="Conflictos")

In [4]:
def seleccionar_equipo(jugadores, conflictos, presupuesto_disponible):
    arqueros = []
    defensas = []
    mediocampistas = []
    delanteros = []
    equipo_total = []
    equipo_total_index = []
    costo_total = 0
    valoracion_total = 0
    for _ in range(3):
        jugadores_arqueros = jugadores[['Nombre', 'Arquero', 'Costo']]
        jugador_aleatorio = jugadores_arqueros.sample(n=1).iloc[0]
        if jugador_aleatorio['Nombre']  not in arqueros and costo_total + jugador_aleatorio['Costo'] <= presupuesto_disponible:
            arqueros.append(jugador_aleatorio['Nombre'])
            equipo_total.append(jugador_aleatorio['Nombre'])
            equipo_total_index.append(jugadores[jugadores['Nombre'] == jugador_aleatorio['Nombre']].index[0])
            valoracion_total += jugador_aleatorio['Arquero'] 
            costo_total += jugador_aleatorio['Costo']
    for _ in range(10):
        jugadores_defensor = jugadores[['Nombre', 'Defensa', 'Costo']]
        jugador_aleatorio = jugadores_defensor.sample(n=1).iloc[0]
        if jugador_aleatorio['Nombre'] not in defensas and costo_total + jugador_aleatorio['Costo'] <= presupuesto_disponible:
            defensas.append(jugador_aleatorio['Nombre'])
            equipo_total.append(jugador_aleatorio['Nombre'])
            equipo_total_index.append(jugadores[jugadores['Nombre'] == jugador_aleatorio['Nombre']].index[0])
            valoracion_total += jugador_aleatorio['Defensa'] 
            costo_total += jugador_aleatorio['Costo']
    for _ in range(12):
        jugadores_mediocampista = jugadores[['Nombre', 'Mediocampista', 'Costo']]
        jugador_aleatorio = jugadores_mediocampista.sample(n=1).iloc[0]
        if jugador_aleatorio['Nombre']  not in mediocampistas and costo_total + jugador_aleatorio['Costo'] <= presupuesto_disponible:
            mediocampistas.append(jugador_aleatorio['Nombre'])
            equipo_total.append(jugador_aleatorio['Nombre'])
            equipo_total_index.append(jugadores[jugadores['Nombre'] == jugador_aleatorio['Nombre']].index[0])
            valoracion_total += jugador_aleatorio['Mediocampista'] 
            costo_total += jugador_aleatorio['Costo']
    for _ in range(8):
        jugadores_delantero = jugadores[['Nombre', 'Delantero', 'Costo']]
        jugador_aleatorio = jugadores_delantero.sample(n=1).iloc[0]
        if jugador_aleatorio['Nombre']  not in delanteros and costo_total + jugador_aleatorio['Costo'] <= presupuesto_disponible:
            delanteros.append(jugador_aleatorio['Nombre'])
            equipo_total.append(jugador_aleatorio['Nombre'])
            equipo_total_index.append(jugadores[jugadores['Nombre'] == jugador_aleatorio['Nombre']].index[0])
            valoracion_total += jugador_aleatorio['Delantero'] 
            costo_total += jugador_aleatorio['Costo']
    conflictos_resultantes = {}
    for i in equipo_total:
        jugador_1 = i
        for j in equipo_total_index:
            jugador_2 = j
            if conflictos.at[jugador_2, jugador_1] == 1:
                conflictos_resultantes[(jugador_1, jugador_2)] = "quizas"
            elif conflictos.at[jugador_2, jugador_1] == 1 == 2:
                conflictos_resultantes[(jugador_1, jugador_2)] = "no"
            else:
                conflictos_resultantes[(jugador_1, jugador_2)] = "si"


    
    return {"MejorEquipo": {
        "Arqueros": arqueros,
        "Defensas": defensas,
        "Mediocampistas": mediocampistas,
        "Delanteros": delanteros
        }, 
    "Valoracion": valoracion_total, "Costo": costo_total, "Conflictos": conflictos_resultantes}

In [5]:
#Se demora aproximadamente 40 segundos en correr, por favor tengan paciencia
start_time_1_1 = time.time()
equipos_generados = []
for _ in range(1000):
    equipo = seleccionar_equipo(datos, conflictos, presupuesto)
    equipos_generados.append(equipo)
end_time_1_1 = time.time()

In [6]:
start_time_1_2 = time.time()
mejor_solucion = min(equipos_generados, key=lambda x: (x['Costo'] - x['Valoracion'] + sum(1000 if tipo == "quizas" else 0 for tipo in x['Conflictos'].values())
        -+ sum(5000000000000000 if tipo == "no" else 0 for tipo in x['Conflictos'].values())
))
end_time_1_2 = time.time()
duration_1 = (end_time_1_1 + end_time_1_2) - (start_time_1_1)

In [7]:
print("Jugadores seleccionados:")
for j in ["Arqueros", "Defensas", "Mediocampistas", "Delanteros"]:
    print(f"    {j}: {mejor_solucion['MejorEquipo'][j]}")
print(f"Valoración total del equipo: {mejor_solucion['Valoracion']}")
print(f"Costo total de la plantilla: {mejor_solucion['Costo']}")
#Contamos conflictos
conflictos_tipo_1 = sum(1 for value in mejor_solucion['Conflictos'].values() if value == 'quizas')
conflictos_tipo_2 = sum(1 for value in mejor_solucion['Conflictos'].values() if value == 'no')


print(f"Conflictos: \n  Tipo 1: {conflictos_tipo_1} \n  Tipo 2: {conflictos_tipo_2}")

Jugadores seleccionados:
    Arqueros: ['Jugador 201', 'Jugador 150', 'Jugador 288']
    Defensas: ['Jugador 86', 'Jugador 55', 'Jugador 178', 'Jugador 206', 'Jugador 230', 'Jugador 134', 'Jugador 208', 'Jugador 321', 'Jugador 166', 'Jugador 252']
    Mediocampistas: ['Jugador 214', 'Jugador 271', 'Jugador 21', 'Jugador 277', 'Jugador 31', 'Jugador 319', 'Jugador 166', 'Jugador 49', 'Jugador 288', 'Jugador 34', 'Jugador 8']
    Delanteros: ['Jugador 138', 'Jugador 49', 'Jugador 28', 'Jugador 166', 'Jugador 291', 'Jugador 200', 'Jugador 103', 'Jugador 319']
Valoración total del equipo: 1655
Costo total de la plantilla: 21873
Conflictos: 
  Tipo 1: 93 
  Tipo 2: 0



*(b) (10 puntos) Diseñe un modelo de optimización lineal en variables binarias, prográmelo y resuélvalo en Python/GUROBI, para encontrar la conformación de la plantilla de jugadores (33 posiciones), garantizando que no existan conflictos y que no se sobrepasa el presupuesto disponible. Compare esta solución con la obtenida en la parte (a).*

In [8]:
#información retirada por una conversación con ChatGPT prompt: "¿Cómo incluyo en mi codigo el API key [Numero de Key] para Gurobi?" 
from gurobipy import *
import pandas as pd
import time
import numpy as np
Env().setParam('CloudAPIKey', '52088e4e-6c08-4d26-96be-91bdd29abd59')

Restricted license - for non-production use only - expires 2024-10-28
No parameters matching 'CloudAPIKey' found


In [9]:
#Linea tomada de ayudantias
modelo = Model("modelo_futbol")

Restricted license - for non-production use only - expires 2024-10-28


In [10]:
## Definimos variables 
# Número de jugadores
num_jugadores = 325

# Número de posiciones
num_posiciones = 4

# Número de jugadores por posición
num_arqueros = 3
num_defensas = 10
num_mediocampistas = 12
num_delanteros = 8

# Presupuesto disponible
presupuesto = 22000

datos = pd.read_excel("Datos.xlsx", sheet_name="Características")
conflictos = pd.read_excel("Datos.xlsx", sheet_name="Conflictos")

In [11]:
#Variables de desición definidas

var_desicion = {}
for i in range(num_jugadores):
    for j in ["Arquero", "Defensa", "Mediocampista", "Delantero"]:
        var_desicion[i, j] = modelo.addVar(vtype=GRB.BINARY, lb=0, name=f"x-{i}-{j}")
#Funcion Objetivo
modelo.setObjective(
    quicksum(
        datos.loc[i, "Arquero"] * var_desicion[i, "Arquero"] +
        datos.loc[i, "Defensa"] * var_desicion[i, "Defensa"] +
        datos.loc[i, "Mediocampista"] * var_desicion[i, "Mediocampista"] +
        datos.loc[i, "Delantero"] * var_desicion[i, "Delantero"]
        for i in range(num_jugadores)
    ),
    GRB.MAXIMIZE
)

#Restricciones

modelo.addConstrs(
    quicksum(var_desicion[i, "Arquero"] for i in range(num_jugadores)) == num_arqueros
    for j in range(num_posiciones)
)
modelo.addConstrs(
    quicksum(var_desicion[i, "Defensa"] for i in range(num_jugadores)) == num_defensas
    for j in range(num_posiciones)
)
modelo.addConstrs(
    quicksum(var_desicion[i, "Mediocampista"] for i in range(num_jugadores)) == num_mediocampistas
    for j in range(num_posiciones)
)
modelo.addConstrs(
    quicksum(var_desicion[i, "Delantero"] for i in range(num_jugadores)) == num_delanteros
    for j in range(num_posiciones)
)
num_conflictos_total = modelo.addVar(vtype=GRB.INTEGER, lb=0, name="num_conflictos_total")
for i in range(num_jugadores):
    for j in range(i+1, num_jugadores):
        if not np.isnan(conflictos.at[i, f"Jugador {j}"]):
            if conflictos[i, j] == 1:
                modelo.addConstr(var_desicion[i, "Arquero"] + var_desicion[j, "Arquero"] <= 1)
                modelo.addConstr(var_desicion[i, "Defensa"] + var_desicion[j, "Defensa"] <= 1)
                modelo.addConstr(var_desicion[i, "Mediocampista"] + var_desicion[j, "Mediocampista"] <= 1)
                modelo.addConstr(var_desicion[i, "Delantero"] + var_desicion[j, "Delantero"] <= 1)
                modelo.addConstr(num_conflictos_total + 1 == num_conflictos_total)
            elif conflictos[i, j] == 2:
                modelo.addConstr(var_desicion[i, "Arquero"] + var_desicion[j, "Arquero"] <= 1)
                modelo.addConstr(var_desicion[i, "Defensa"] + var_desicion[j, "Defensa"] <= 1)
                modelo.addConstr(var_desicion[i, "Mediocampista"] + var_desicion[j, "Mediocampista"] <= 1)
                modelo.addConstr(var_desicion[i, "Delantero"] + var_desicion[j, "Delantero"] <= 1)
                modelo.addConstr(num_conflictos_total + 1 == num_conflictos_total)


In [12]:
#Respuesta óptima
start_time_2 = time.time()
modelo.optimize()
end_time_2 = time.time()
duration_2 = end_time_2 - start_time_2

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 1301 columns and 5200 nonzeros
Model fingerprint: 0x774b0865
Variable types: 0 continuous, 1301 integer (1300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+01]
Found heuristic solution: objective 1695.0000000
Presolve removed 15 rows and 1200 columns
Presolve time: 0.00s
Presolved: 1 rows, 101 columns, 101 nonzeros
Found heuristic solution: objective 3223.0000000
Variable types: 0 continuous, 101 integer (12 binary)
Found heuristic solution: objective 3235.0000000

Root relaxation: objective 3.245000e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Un

In [13]:
#Creamos combinación óptima de calificaciones
clasificaciones = {}
#Iteramos por las mejores respuestas
for j in ["Arquero", "Defensa", "Mediocampista", "Delantero"]:
    clasificaciones[j] = []
if modelo.status == GRB.OPTIMAL:
    print("Jugadores seleccionados:")
    for i in range(num_jugadores):
        for j in ["Arquero", "Defensa", "Mediocampista", "Delantero"]:
            if var_desicion[i, j].x > 0.5:
                clasificaciones[j].append(i)
    for j in ["Arquero", "Defensa", "Mediocampista", "Delantero"]:
        print(f"    Posición {j}: {clasificaciones[j]}")
    costo_2_expr = modelo.getAttr('X', var_desicion).sum()
    costo_2 = int(costo_2_expr.getValue())
    print(f"Valoración total del equipo: {modelo.objVal}")
    print(f"Costo total de la plantilla: {costo_2} pesos")
    print(f"Número total de conflictos en el mejor caso: {int(num_conflictos_total.x)}")
else:
    print("No se encontró una solución óptima.")

Jugadores seleccionados:
    Posición Arquero: [177, 245, 298]
    Posición Defensa: [15, 26, 31, 50, 68, 70, 166, 306, 322, 323]
    Posición Mediocampista: [10, 94, 105, 128, 159, 161, 162, 197, 243, 281, 288, 315]
    Posición Delantero: [19, 49, 76, 83, 150, 155, 178, 206]
Valoración total del equipo: 3245.0
Costo total de la plantilla: 33 pesos
Número total de conflictos en el mejor caso: 0


### Comparaciones y conclusiones

In [14]:
import re
arqueros_1 = mejor_solucion['MejorEquipo']["Arqueros"]
defensa_1 = mejor_solucion['MejorEquipo']["Defensas"]
mediocampo_1 = mejor_solucion['MejorEquipo']["Mediocampistas"]
delanteros_1 = mejor_solucion['MejorEquipo']["Delanteros"]
conflictos_tipo_1_1 = sum(1 for value in mejor_solucion['Conflictos'].values() if value == 'quizas')
conflictos_tipo_2_1 = sum(1 for value in mejor_solucion['Conflictos'].values() if value == 'no')
valor_1 = mejor_solucion['Valoracion']
costo_1 = mejor_solucion['Costo']
conflictos_total_1 = conflictos_tipo_1 + conflictos_tipo_2
conflictos_total_2 = int(num_conflictos_total.x)

if modelo.status == GRB.OPTIMAL:
    arqueros_2 = clasificaciones["Arquero"]
    defensa_2 = clasificaciones["Defensa"]
    mediocampo_2 = clasificaciones["Mediocampista"]
    delanteros_2 = clasificaciones["Delantero"]
    valoracion_2 = modelo.objVal

    #Comparamos variables
    arqueros_comun = 0
    delanteros_comun = 0
    defensa_comun = 0
    mediocampo_comun = 0

    for x, y in zip(arqueros_1, arqueros_2):
        num_x = int(re.findall("\d+", x)[0])
        if num_x == y:
            arqueros_comun += 1
    for x, y in zip(delanteros_1, delanteros_1):
        num_x = int(re.findall("\d+", x)[0])
        if num_x == y:
            delanteros_comun += 1
    for x, y in zip(defensa_1, defensa_2):
        num_x = int(re.findall("\d+", x)[0])
        if num_x == y:
            defensa_comun += 1
    for x, y in zip(mediocampo_1, mediocampo_2):
        num_x = int(re.findall("\d+", x)[0])
        if num_x == y:
            mediocampo_comun += 1
    

In [17]:
if modelo.status == GRB.OPTIMAL:
#Equipos formados
    print("Considencias entre los equipos: ")
    print(f"   Arqueros en común: {arqueros_comun}\n   Delanteros en común: {delanteros_comun}\n   Defensas en común: {defensa_comun}\n   Mediocampos en común:{mediocampo_comun}")
    
    
    print("Comparaciones entre los equipos y optimización:")
    #Valoración
    gana = {"a": 0, "b": 0}
    if valor_1 > valoracion_2:
        print(" El equipo formado en (a) da mejor valor total")
        gana["a"] += 1
    else:
        print(" El equipo formado en (b) da mejor valor total")
        gana["b"] += 1
    #Costos
    integer_value = int(costo_2)
    if costo_2 > costo_1:
        print(" El equipo formado en (a) tiene menor costo")
        gana["a"] += 1
    else:
        print(" El equipo formado en (b) tiene menor costo")
        gana["b"] += 1
    #Conflictos
    if conflictos_total_1 > conflictos_total_2:
        print(" El equipo formado en (b) tiene menos conflictos")
        gana["b"] += 1
    else:
        print(" El equipo formado en (a) tiene menos conflictos")
        gana["a"] += 1
    # Tiempo
    if duration_1 > duration_2:
        print(" Optimizar el problema demora menos en (b)")
        gana["b"] += 1
    else:
        print(" Optimizar el problema demora menos en (a)")
        gana["a"] += 1
    #Conclusión
    print("Por lo tanto, concluimos que: ")
    if gana["a"]  > gana["b"]:
        print(" Es mejor usar la estrategia (a)")
    elif gana["b"] > gana["a"]:
        print(" Es mejor usar la estrategia (b)")
    else:
        print(" Es igual usar (a) o (b)")

else: 
    print("No se pueden comparar porque algo falló en (b)")


Considencias entre los equipos: 
   Arqueros en común: 0
   Delanteros en común: 0
   Defensas en común: 0
   Mediocampos en común:0
Comparaciones entre los equipos y optimización:
 El equipo formado en (b) da mejor valor total
 El equipo formado en (b) tiene menor costo
 El equipo formado en (b) tiene menos conflictos
 Optimizar el problema demora menos en (b)
Por lo tanto, concluimos que: 
 Es mejor usar la estrategia (b)
